**Task 1: Multiple Inheritance**

Multiple inheritance is a feature of object‑oriented programming where a class can inherit attributes and methods from more than one parent class. Python supports multiple inheritance directly, but it requires a clear understanding of how names are resolved to avoid unexpected behaviour.

### 1. Basic Syntax

In [1]:

class Parent1:
    def method1(self):
        print("Parent1.method1")

class Parent2:
    def method2(self):
        print("Parent2.method2")

class Child(Parent1, Parent2):
    def child_method(self):
        print("Child.child_method")

c = Child()
c.method1()       # from Parent1
c.method2()       # from Parent2
c.child_method()


Parent1.method1
Parent2.method2
Child.child_method


### 2. Method Resolution Order (MRO)
When a method is called on an object, Python must decide which class’s method to use, especially if multiple parents define the same method. The **MRO** is the order in which base classes are searched. Python uses the **C3 linearization algorithm** (also called C3 superclass linearization) to compute a consistent MRO.

- You can view the MRO of a class using `ClassName.__mro__` or `ClassName.mro()`.
- The MRO ensures that:
  - A class always appears before its parents.
  - If two parents inherit from a common ancestor, that ancestor appears only once and after both parents.

**Example:**

In [2]:
class A:
    def who(self):
        print("A")

class B(A):
    def who(self):
        print("B")

class C(A):
    def who(self):
        print("C")

class D(B, C):
    pass

print(D.__mro__)
# (<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)

d = D()
d.who()   # prints "B" because B is first in the MRO after D

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
B


### 3. The `super()` Function in Multiple Inheritance
`super()` returns a proxy object that delegates method calls to the next class in the MRO. It is especially useful in cooperative multiple inheritance to ensure that all parent classes are initialised correctly.

**Example with constructors:**

In [3]:
class A:
    def __init__(self):
        print("A.__init__")
        super().__init__()   # calls next in MRO

class B:
    def __init__(self):
        print("B.__init__")
        super().__init__()

class C(A, B):
    def __init__(self):
        print("C.__init__")
        super().__init__()

c = C()
# Output:
# C.__init__
# A.__init__
# B.__init__

C.__init__
A.__init__
B.__init__


Notice that `super()` in `A` calls `B.__init__` because the MRO of `C` is `(C, A, B, object)`. This cooperative design allows every class in the hierarchy to be initialised, even if they are not directly related.


### 4. The Diamond Problem and How Python Solves It
The **diamond problem** occurs when a class inherits from two classes that share a common ancestor. Without a proper resolution order, the ancestor’s methods might be called twice or ambiguously.

Python’s MRO guarantees that the common ancestor is called only once and after all its derived classes. In the example above, `A` appears only once in the MRO, and both `B` and `C` are visited before it.

### 5. Potential Pitfalls
- **Name clashes:** If two parent classes define the same method name, the one that appears first in the MRO wins. This can lead to surprising behaviour if you rely on both implementations.
- **Complex MRO:** Deep inheritance hierarchies can make the MRO difficult to predict. Always check `__mro__` if unsure.
- **Incorrect use of `super()`:** For `super()` to work cooperatively, all classes in the hierarchy must use `super()` consistently, even if they are at the top of the hierarchy (where `super()` calls `object.__init__`).

### 6. Practical Use: Mixins
Multiple inheritance is often used to create **mixins** – small classes that provide specific, reusable functionality. A mixin is not meant to stand alone; it is combined with a main class to add behaviour.


In [7]:
class JSONMixin:
    def to_json(self):
        import json
        return json.dumps(self.__dict__)

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

class Employee(JSONMixin, Person):
    pass

e = Employee("Alice", 30)
print(e.to_json())   # {"name": "Alice", "age": 30}

{"name": "Alice", "age": 30}


### Summary
Multiple inheritance in Python is a powerful tool when used with care. Understanding MRO and cooperative `super()` calls is essential to avoid subtle bugs. It shines in mixin‑based designs, allowing you to compose behaviour from small, focused classes.
